In [0]:
%load_ext autoreload
%autoreload 2

import matplotlib
from matplotlib import pyplot
pyplot.ion()
pyplot.style.use('dark_background')

import torch as th
from rl.core.algs.dqn import *

In [0]:
from rl.core.algs import experiment
from rl.core.algs import model
from rl.core.envs import grid_world

BATCH_SIZE = 32
TRAIN_START_STEP = 3 * BATCH_SIZE
NUM_TRAIN_STEPS = 10000 # 17
REPLAY_SIZE = 100000
TARGET_NETWORK_UPDATE_FREQ = 500


env = environment.GridWorldWrapped(grid_shape=(4, 4, 1))
conv_specs = {
  model.ConvSpec(32, 4, 1),
}
fc_specs = (64,)
model_params = {
  'conv_specs': conv_specs,
  'fc_specs': fc_specs,
}

EPS_DECAY = util.Schedule((0, TRAIN_START_STEP, NUM_TRAIN_STEPS / 1, NUM_TRAIN_STEPS),
                          (1, 1, 0.0, 0.0))
EPS_RANDOM = util.Schedule((0,), (1.0,))
adam_params = {
  'fn': th.optim.Adam,
  'kwargs': {
    'lr': 0.005,
  }
}
rmsprop_params = {
  'fn': th.optim.RMSprop,
  'kwargs': {
    'lr': 0.00025,
    'momentum': 0.95,
    'alpha': 0.95,
    'eps': 0.01,
  }
}

policy_params = {
    'opt_params': rmsprop_params,
    'eps_sched': EPS_DECAY,  # TODO: change
    'target_update_freq': TARGET_NETWORK_UPDATE_FREQ,
}
exp = experiment.Experiment(env, model.QNetwork, model_params,
                            Dqn, policy_params)

print(exp.policy.model)

rb = ReplayBuffer(REPLAY_SIZE)
for step in range(TRAIN_START_STEP):
  sars = env.step(exp.policy.get_action)
  rb.add(sars)

eps_rew = 0
for step in range(TRAIN_START_STEP, NUM_TRAIN_STEPS):
  sars = env.step(exp.policy.get_action)
  rb.add(sars)
    
  experience_batch = rb.sample(BATCH_SIZE)
  m = exp.policy.update(experience_batch)
  
  eps_rew += sars.r
  if sars.s1 is None:
    m['r_per_eps'] = eps_rew
    eps_rew = 0
  if step % 100 == 0 or sars.s1 is None:
    for k, v in m.items():
      exp.plt.add_data(k, step, v)
      
# Expect about r_per_eps=-50 in pure random agent.

(1, 1, 32)
QNetwork(
  (convs): Sequential(
    (0): Conv2d(1, 32, kernel_size=(4, 4), stride=(1, 1))
    (1): ReLU()
  )
  (fc): Sequential(
    (0): Linear(in_features=32, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=4, bias=True)
  )
)


In [0]:
opt = grid_world.GridWorld((4, 4)).optimal_expected_reward()
print(f'optimal expected r: {opt}')

optimal expected r: 1.6666666666666665


In [0]:
env.show_model_policy(exp.policy.model)

[[ 0.          0.01253885 -0.8965975  -1.86765313]
 [-0.03269446 -0.84576964 -1.79328823 -2.52134728]
 [-0.93795955 -1.78422523 -2.70635748 -3.40485382]
 [-1.94635844 -2.70565581 -3.62358236 -4.38774967]]
[['⟲' '←' '←' '←']
 ['↑' '←' '←' '←']
 ['↑' '←' '←' '←']
 ['↑' '←' '←' '←']]
[[-0.00589567  0.          0.13667262 -0.92200089]
 [-0.94206083  0.02738249 -0.83347172 -1.80942607]
 [-1.78275788 -1.00875008 -1.81086373 -2.65875626]
 [-2.84708357 -1.97399151 -2.77122307 -3.74289608]]
[['→' '⟲' '←' '←']
 ['→' '↑' '←' '←']
 ['↑' '↑' '←' '←']
 ['↑' '↑' '↑' '↑']]
[[-1.04315972 -0.02504301  0.          0.02958971]
 [-1.91494203 -0.9430964   0.00940329 -0.98650539]
 [-2.92589617 -1.9303515  -0.97228563 -1.90941215]
 [-3.87441969 -2.97553015 -1.91040087 -2.74810123]]
[['→' '→' '⟲' '←']
 ['→' '→' '↑' '←']
 ['↑' '→' '↑' '←']
 ['↑' '↑' '↑' '↑']]
[[-1.97671604 -1.02534652  0.05006999  0.        ]
 [-2.89166856 -1.97657633 -0.96848488  0.008448  ]
 [-3.8940444  -2.82773399 -2.01320696 -1.02480412]
 

In [0]:
# Debug the action-value function for a given state.
s = [[0, 0, 0, 0],
     [0, 0, 0, 0],
     [0, 0, 1, 0],
     [0, 0, 2, 0]]
s = np.expand_dims(np.expand_dims(s, 0), -1)
print(exp.policy.model(util.to_tensor(s)))
print(exp.policy.target_model(util.to_tensor(s)))

tensor([[-2.1735, -2.0206,  0.0484, -1.9368]], device='cuda:0',
       grad_fn=<AddmmBackward>)
tensor([[-2.1070, -2.0194, -0.0236, -2.0042]], device='cuda:0',
       grad_fn=<AddmmBackward>)


In [0]:
# Examine the replay buffer.
terminals = rb.filter_by(sars_filter(r=0))
print(len(terminals))

# action-value at 's' was wrong. Search the replay buffer for occurrences of
# the correct transition. It's likely missing, which led the NN to overfit
# on missing data.
trans_s = rb.filter_by(sars_filter(s=s))
trans_s_a = rb.filter_by(sars_filter(s=s,a=2))
print(f'Num with (s): {len(trans_s)}. Num with (s, a): {len(trans_s_a)}')


1691
Num with (s): 87. Num with (s, a): 59


In [0]:
env.visualize(exp.policy, steps=20)

array([[0, 0, 0, 0],
       [0, 0, 2, 0],
       [0, 0, 0, 0],
       [0, 1, 0, 0]])
array([[0, 0, 0, 0],
       [0, 0, 2, 0],
       [0, 0, 0, 0],
       [0, 0, 1, 0]])
array([[0, 0, 0, 0],
       [0, 0, 2, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 0]])
array([[0, 0, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]])
array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 2, 1, 0],
       [0, 0, 0, 0]])
array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 0, 0]])
array([[0, 0, 0, 0],
       [0, 2, 0, 1],
       [0, 0, 0, 0],
       [0, 0, 0, 0]])
array([[0, 0, 0, 0],
       [0, 2, 1, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]])
array([[0, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]])
array([[0, 0, 1, 0],
       [0, 0, 0, 0],
       [0, 2, 0, 0],
       [0, 0, 0, 0]])
array([[0, 1, 0, 0],
       [0, 0, 0, 0],
       [0, 2, 0, 0],
       [0, 0, 0, 0]])
array([[0, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 2, 0, 0],
  